In [3]:
import pandas as pd

In [1]:
def primera_llamada(name):

    df = pd.read_csv('ibex35.csv')

    name = df[df['empresa'] == name]

    name = name [['date','open','high','low','close','adj close']]

    name['date'] = pd.to_datetime(name['date'], format="%Y-%m-%d")

    name.sort_values(by='date',ascending= True, inplace=True)

    fecha_inicio = '2021-01-01'
    fecha_fin = '2024-03-01'

    name = pd.DataFrame(name[(name['date'] >= fecha_inicio) & (name['date'] <= fecha_fin)]).reset_index(drop=True)

    return name

In [4]:
acerinox =primera_llamada('Acerinox')

acerinox

,date,open,high,low,close,adj close
0,2021-01-04,9.180,9.432,9.142,9.294,7.683368
1,2021-01-05,9.310,9.334,9.214,9.312,7.698248
2,2021-01-06,9.334,9.546,9.304,9.538,7.885085
3,2021-01-07,9.590,9.840,9.550,9.820,8.118213
4,2021-01-08,9.900,9.900,9.670,9.712,8.028930
...,...,...,...,...,...,...
807,2024-02-26,10.245,10.255,10.130,10.230,10.230000
808,2024-02-27,10.260,10.400,10.250,10.400,10.400000
809,2024-02-28,10.365,10.395,10.175,10.200,10.200000
810,2024-02-29,10.245,10.260,10.095,10.135,10.135000


In [21]:

def preparar_modelo(DF):


    DF_filt = DF[['date','close']]

    DF_filt.close = DF.close.round(2)

    DF_filt= DF_filt.reset_index(drop=True)

    new_df = pd.DataFrame(columns=['d-4', 'd-3', 'd-2', 'd-1', 'do'])

    for i in range(len(new_df.columns)):
        new_df[new_df.columns[i]] = DF_filt['close'].shift(-i)

    new_df['date']= DF_filt.date

    new_df.insert(0, 'date', new_df.pop('date'))

    new_df.dropna(axis=0,inplace=True)

    return new_df



In [22]:
new_df = preparar_modelo(acerinox)

new_df

C:\Users\javie\AppData\Local\Temp\ipykernel_420\4040868792.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_filt.close = DF.close.round(2)


,date,d-4,d-3,d-2,d-1,do
0,2021-01-04,9.29,9.31,9.54,9.82,9.71
1,2021-01-05,9.31,9.54,9.82,9.71,9.70
2,2021-01-06,9.54,9.82,9.71,9.70,9.78
3,2021-01-07,9.82,9.71,9.70,9.78,9.81
4,2021-01-08,9.71,9.70,9.78,9.81,9.82
...,...,...,...,...,...,...
803,2024-02-20,10.16,10.28,10.36,10.26,10.23
804,2024-02-21,10.28,10.36,10.26,10.23,10.40
805,2024-02-22,10.36,10.26,10.23,10.40,10.20
806,2024-02-23,10.26,10.23,10.40,10.20,10.14


In [36]:

def modelo_train(new_df):

    # Importar herramientas:

    from sklearn.model_selection import train_test_split as tts 
    from sklearn.ensemble import RandomForestRegressor as RFR 
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP
    from sklearn.metrics import mean_squared_error as mse

    # Separar X e y:

    X = new_df.drop(['do','date'],axis=1)
    y = new_df.do

    # Ejecutar tts:

    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # Iniciar Modelos

    rfr=RFR()
    omp= OMP()

    # Entrenar

    rfr.fit(X_train, y_train)
    omp.fit(X_train, y_train)
     
    # Predecir

    y_pred_rfr=rfr.predict(X_test)
    y_pred_omp=omp.predict(X_test)

    # Mostrar rmse de cada modelo   
   
    return 'RMSE RFR ======',mse(y_test, y_pred_rfr, squared=False),'REMS OMP ======',mse(y_test, y_pred_omp, squared=False)

In [37]:
modelo_train(new_df)

('RMSE RFR ======',
 0.22585524808252655,
 'REMS OMP ======',
 0.20798462361777012)